In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

In [2]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

x = torch.tensor([[0, 0, 0, 0], [1, 1, 1, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

y = torch.tensor([[3.0], [1.0]]) # ranks

data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)

In [3]:
data.num_nodes, data.num_edges, data.num_node_features, data.has_isolated_nodes(), data.has_self_loops(), data.is_directed()

(2, 6, 4, False, False, True)

In [4]:
data.keys()

['y', 'x', 'edge_index']

In [5]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]

In [6]:
adjacency

tensor([[0., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])

In [7]:
data.x

tensor([[0., 0., 0., 0.],
        [1., 1., 1., 1.]])

In [8]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.x.shape[0], 16)
        self.conv2 = GCNConv(16, 16)
        self.out = torch.nn.Linear(16, data.x.shape[0])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = gap(x, None)
        return self.out(x)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
# data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.y)
    print("Loss:", loss)
    loss.backward()
    optimizer.step()

../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [0,0,0], thread: [1,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [0,0,0], thread: [2,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

test_data = Data(x=x, edge_index=data.edge_index)

model(test_data)

tensor([[2.1558, 2.2071]], grad_fn=<AddmmBackward0>)

In [ ]:
data.edge_index

tensor([[0, 0, 0, 1, 2, 3],
        [1, 2, 3, 0, 0, 0]])